In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams, cm
from scipy import integrate, fft
from scipy.integrate import odeint,complex_ode
from scipy.linalg import eigh
from mpl_toolkits.mplot3d import Axes3D
from tabulate import tabulate
from odeintw import odeintw
import time
import itertools

base_path = '/Users/max/Documents/HHU Mathe/Bachelorarbeit/Tex/images'
# Plotting settings
plt.rcParams['figure.figsize'] = [24, 12]
plt.rcParams.update({'font.size': 14})

In [ ]:
##### Zeitliche Diskretisierung #####
t_range = 2*np.pi
m = 500
t = np.linspace(0,t_range,m)

##### Räumliche Diskretisierung #####
L2 = 40
n = 1000
x_pre = np.linspace(-L2/2,L2/2,n+1)
x = x_pre[:n] 

##### Anfangsbedingung (Soliton) #####
N = 2
u0 = N/np.cosh(x)

In [ ]:
%run -i Functions.py

X = NLS_POD_Offline_1(u0, x, t)
X_N = (1j)*np.power(np.abs(X),2)*X

ops = NLS_POD_Offline_2_3(X, 2)
X_pod = NLS_POD_Online(u0, t, ops)
ops = NLS_POD_DEIM_Offline_2_3_4(X, X_N, 2, 2)
X_deim = NLS_POD_DEIM_Online(u0, t, ops)

In [ ]:
##### Energieverteilung der Singulärwerte/-vektoren #####

Psi,Sigma,PhiT = np.linalg.svd(X,full_matrices=0)

max_r = 5
data = [["r"],["Energieanteil"]]
for r in range(1,max_r+1):
    data = np.hstack((data, [[r],[f'{round((np.sum(np.power(Sigma[:r],2))/np.sum(np.power(Sigma,2)))*100,5)}%']]))
    
print(f'Gesamtsumme der Energie des Systems: {np.sum(np.power(Sigma, 2))}')    
#display table
print(tabulate(data, tablefmt="fancy_grid"))

In [ ]:
##### PLOTTING #####
%run -i Functions.py
#%matplotlib notebook

plt.rcParams.update({'font.size': 10})
fig = plt.figure(figsize=(15,12))
#fig.subplots_adjust(wspace=.1,hspace=.1)

x2 = x[300:700]
X_plot = X[300:700,:]
X_pod_plot = X_pod[300:700,:]
X_deim_plot = X_deim[300:700,:]

ax1 = fig.add_subplot(2, 2, 3, projection='3d')
ax1.view_init(elev=25, azim=130)
for tt in range(len(t)):
    ax1.plot(x2,t[tt]*np.ones_like(x2),np.abs(X_pod_plot[:,tt]),color='k',linewidth=0.05)
ax1.set_xlabel('x')
ax1.set_ylabel('t')
ax1.set_zlabel('u')
#ax1.set_xlim(-8,8)
ax1.set_title('(c) POD ROM (r=3)', fontdict={'fontsize':'14'})

ax2 = fig.add_subplot(2, 2, 4, projection='3d')
ax2.view_init(elev=25, azim=130)
for tt in range(len(t)):
    ax2.plot(x2,t[tt]*np.ones_like(x2),np.abs(X_deim_plot[:,tt]),color='k',linewidth=0.05)
ax2.set_xlabel('x')
ax2.set_ylabel('t')
ax2.set_zlabel('u')
#ax2.set_xlim(-8,8)
ax2.set_title('(d) POD DEIM ROM (r=p=3)', fontdict={'fontsize':'14'})

ax3 = fig.add_subplot(2, 2, 1, projection='3d')
ax3.view_init(elev=25, azim=130)
for tt in range(len(t)):
    ax3.plot(x2,t[tt]*np.ones_like(x2),np.abs(X_plot[:,tt]),color='k',linewidth=0.05)
ax3.set_xlabel('x')
ax3.set_ylabel('t')
ax3.set_zlabel('u')
#ax1.set_xlim(-8,8)
ax3.set_title('(a)FOM', fontdict={'fontsize':'14'})

XI,Sigma_NL,PhiT_NL = np.linalg.svd(X_N,full_matrices=0)
p = 3
P = DEIM_Methode(XI, p)
indexes = np.where(np.any(P>0, axis=1))[0]

ax4 = fig.add_subplot(2, 2, 2)
color_list = ['b','g','r']
for jj in range(p):
    ax4.plot(x,np.real(Psi[:,jj]),color=color_list[jj],linewidth=2, \
             label=r'$\psi_{}$'.format(jj+1))
    ax4.axvline(x[indexes[jj]], color='k')
ax4.set_xlim(-4,4)
ax4.set_title('(b) Singulärvektoren und Interpolationspunkte', fontdict={'fontsize':'14'})
plt.legend(loc=1, prop={'size': 14})

fig.savefig(f'{base_path}/DEIM_Analysis.png', bbox_inches='tight', dpi=600)
fig.savefig(f'{base_path}/DEIM_Analysis.svg', bbox_inches='tight', dpi=600)

plt.show()

In [ ]:
%run -i Functions.py
##### Bestimme Zeiten für POD-DEIM Methode #####

n_arr = [500, 1000, 2000, 4000]
Offline_time_deim = np.zeros((len(n_arr)))
Online_time_deim = np.zeros((len(n_arr)))
Offline_time_pod = np.zeros((len(n_arr)))
Online_time_pod = np.zeros((len(n_arr)))



for j in range(len(n_arr)):
    print(f'Berechne Zeiten für n = {n_arr[j]}')
    
    ##### Räumliche Diskretisierung #####
    n = n_arr[j]
    x_pre = np.linspace(-L2/2,L2/2,n+1)
    x = x_pre[:n]

    ##### Anfangsbedingung (Soliton) #####
    u0 = N/np.cosh(x)

    X_arr, off_1_time = NLS_POD_DEIM_Offline_1_timed(u0, x, t)
    Offline_time_deim[j] = np.sum(np.array(off_1_time))
    operators, off_2_3_4_times = NLS_POD_DEIM_Offline_2_3_4_timed(X_arr[0], X_arr[1], 2, 3)
    Offline_time_deim[j] += np.sum(np.array(off_2_3_4_times))
    X_rom, on_1_2_times = NLS_POD_DEIM_Online_timed(u0, t, operators)
    Online_time_deim[j] = np.sum(np.array(on_1_2_times))
    
    X, off_1_time = NLS_POD_Offline_1_timed(u0, x, t)
    Offline_time_pod[j] = np.sum(np.array(off_1_time))
    operators, off_2_3_times = NLS_POD_Offline_2_3_timed(X, 2)
    Offline_time_pod[j] += np.sum(np.array(off_2_3_times))
    X_rom, on_1_2_times = NLS_POD_Online_timed(u0, t, operators)
    Online_time_pod[j] = np.sum(np.array(on_1_2_times))

In [ ]:
##### PLOTTING #####
plt.rcParams.update({'font.size': 12})
plt.figure(figsize=(15,5))

plt.plot(n_arr, Offline_time_pod, label = "POD Offline", linestyle='dashed', marker='s')
plt.plot(n_arr, Offline_time_deim, label = "DEIM Offline", linestyle='dashed', marker='o') 
plt.plot(n_arr, Online_time_pod, label = "POD Online", linestyle='dashed', marker='s')
plt.plot(n_arr, Online_time_deim, label = "DEIM Online", linestyle='dashed', marker='o') 

plt.xticks(n_arr, n_arr)
plt.yscale('log')
plt.xlabel('n')
plt.ylabel('Zeit in Sekunden')
plt.legend()
plt.savefig(f'{base_path}/DEIM_timings.png', bbox_inches='tight', dpi=600)
plt.savefig(f'{base_path}/DEIM_timings.svg', bbox_inches='tight', dpi=600)
plt.show()

In [ ]:
(Online_time_pod - Online_time_deim)/Online_time_pod

In [ ]:
##### Unterschied in der Offline Zeit des p=3 und 8 POD-DEIM ROMs #####

deim3_offline = 0
deim8_offline = 0

##### Räumliche Diskretisierung #####
n = 1000
x_pre = np.linspace(-L2/2,L2/2,n+1)
x = x_pre[:n]

##### Anfangsbedingung (Soliton) #####
u0 = N/np.cosh(x)

# Dieser Abschnitt kann in der Betrachtung außer Acht gelassen werden, da er unaabhängig von p ist.
X_arr = NLS_POD_DEIM_Offline_1(u0, x, t)

operators3, off_2_3_4_times = NLS_POD_DEIM_Offline_2_3_4_timed(X_arr[0], X_arr[1], 2, 3)
deim3_offline = np.sum(np.array(off_2_3_4_times))
operators8, off_2_3_4_times = NLS_POD_DEIM_Offline_2_3_4_timed(X_arr[0], X_arr[1], 2, 8)
deim8_offline = np.sum(np.array(off_2_3_4_times))

increase_offline = (deim8_offline-deim3_offline)/deim8_offline
print(increase_offline)

In [ ]:
n_arr = [1000, 2000, 4000]
p_range = 6 # p = 3-8

times = np.zeros((len(n_arr), p_range+1))

for i in range(len(n_arr)):
    
    ##### Räumliche Diskretisierung #####
    n = n_arr[i]
    x_pre = np.linspace(-L2/2,L2/2,n+1)
    x = x_pre[:n]

    ##### Anfangsbedingung (Soliton) #####
    u0 = N/np.cosh(x)

    # Dieser Abschnitt kann in der Betrachtung außer Acht gelassen werden, da er unaabhängig von p ist.
    X_arr = NLS_POD_DEIM_Offline_1(u0, x, t)
    
    operators_pod = NLS_POD_Offline_2_3(X_arr[0], 2)
    X_rom, on_1_2_times = NLS_POD_Online_timed(u0, t, operators_pod)
    times[i,0] = np.sum(np.array(on_1_2_times))

    for j in range(1,p_range+1):
        print(f"Berechne Zeit für n = {n_arr[i]}, p = {j+2}...")
        operators = NLS_POD_DEIM_Offline_2_3_4(X_arr[0], X_arr[1], 2, j+2)
        X_rom, on_1_2_times = NLS_POD_DEIM_Online_timed(u0, t, operators)
        times[i,j] = np.sum(np.array(on_1_2_times))

In [ ]:
##### PLOTTING #####
plt.rcParams.update({'font.size': 12})
plt.figure(figsize=(15,6))

marker = itertools.cycle(('.', '>', 'X', 'D', 'P', 'd', '*'))

plt.plot(n_arr, times[:,0], label = f"POD ROM", linestyle='dashed', marker=next(marker))
for i in range(p_range):
    plt.plot(n_arr, times[:,i+1], label = f"DEIM ROM (p={i+3})", linestyle='dashed', marker=next(marker))

plt.xticks(n_arr, n_arr)
plt.yscale('log')
plt.xlabel('n')
plt.ylabel('Zeit in Sekunden')
plt.legend()
plt.savefig(f'{base_path}/DEIM_timings_online.png', bbox_inches='tight', dpi=600)
plt.savefig(f'{base_path}/DEIM_timings_online.svg', bbox_inches='tight', dpi=600)
plt.show()

In [ ]:
##### PLOTTING #####

data = [["p"],["Abnahme"]]
for p in range(3,p_range):
    decrease = (pod_online_time - deim_online_times[p-3])/pod_online_time
    data = np.hstack((data, [[p],[decrease]]))
    
print(f'POD Online Zeit: {pod_online_time}')    
#display table
print(tabulate(data, tablefmt="fancy_grid"))

In [ ]:
%run -i Functions.py

##### Determine Errors for different r's  and p's#####
r_range = 8
p_range = 8
total_error_deim = np.zeros((r_range, p_range))
total_error_pod = np.zeros(r_range)


##### Räumliche Diskretisierung #####
n = 1000
x_pre = np.linspace(-L2/2,L2/2,n+1)
x = x_pre[:n]

##### Anfangsbedingung (Soliton) #####
u0 = N/np.cosh(x)

X_true = Simulate_NLS_FOM(u0, x, t)
X = NLS_POD_Offline_1(u0, x, t)
X_N = (1j)*np.power(np.abs(X),2)*X

for r in range(1,r_range+1):
    print(f'Fehler wird berechnet für r = {r}')
    ops = NLS_POD_Offline_2_3(X, r)
    X_pod = NLS_POD_Online(u0, t, ops)
    for p in range(1,p_range+1):
        ops = NLS_POD_DEIM_Offline_2_3_4(X, X_N, p, r)
        X_deim = NLS_POD_DEIM_Online(u0, t, ops)
        total_error_deim[r-1, p-1] = np.linalg.norm(X_true - X_deim, 'fro')**2
    
    #for tt in range(t.size):
    total_error_pod[r-1] = np.linalg.norm(X_true - X_pod, 'fro')**2 

In [ ]:
##### PLOTTING #####
plt.rcParams.update({'font.size': 12})
plt.figure(figsize=(15,6))
marker = itertools.cycle(('.', '>', 'X', 'D', 'P', 'd', '*'))


x = np.arange(r_range)

plt.plot(x, total_error_pod, label = "POD Fehler", linestyle='dashed', marker='s')
for p in range(p_range):
    plt.plot(x, total_error_deim[:,p], label = f"DEIM (p = {p+1}) Fehler", linestyle='dashed', marker=next(marker))

plt.xticks(x, x+1)
plt.yscale('log')
plt.xlabel('r', fontsize=14)
plt.ylabel(r'$E_{ROM}(r)$', fontsize=14)
plt.legend()
plt.savefig(f'{base_path}/DEIM_error.png', bbox_inches='tight', dpi=600)
plt.savefig(f'{base_path}/DEIM_error.svg', bbox_inches='tight', dpi=600)
plt.show()

In [ ]:
%run -i Functions.py

##### N = 2 Soliton #####
##### Plot Modell- vs Projektionsfehler des POD ROMs #####
r_range = 7
total_error_pod_model = np.zeros(r_range)
total_error_pod_proj = np.zeros(r_range)



##### Räumliche Diskretisierung #####
n = 1000
x_pre = np.linspace(-L2/2,L2/2,n+1)
x = x_pre[:n]

##### Anfangsbedingung (Soliton) #####
N = 2
u0 = N/np.cosh(x)

X_true = Simulate_NLS_FOM(u0, x, t)
X = NLS_POD_Offline_1(u0, x, t)

Psi, Sigma, PhiT = np.linalg.svd(X)

for r in range(1,r_range+1):
    print(f'Fehler wird berechnet für r = {r}')
    Psi_inner = Psi[:,:r]@Psi[:,:r].T
    ops = NLS_POD_Offline_2_3(X, r)
    X_pod = NLS_POD_Online(u0, t, ops)
    total_error_pod_proj[r-1] += np.linalg.norm(X_true[:,tt] - Psi_inner@X_true[:,tt])**2
    total_error_pod_model[r-1] += np.linalg.norm(Psi_inner@X_true[:,tt] - X_pod[:,tt])**2
    

In [ ]:
##### PLOTTING #####
plt.rcParams.update({'font.size': 12})
plt.figure(figsize=(15,7))
marker = itertools.cycle(('.', '>', 'X', 'D', 'P', 'd', '*'))


x = np.arange(r_range)

plt.plot(x, total_error_pod_model, label = "POD Modellierungsfehler", linestyle='dashed', marker='s')
plt.plot(x, total_error_pod_proj, label = "POD Projektionsfehler", linestyle='dashed', marker='s')

plt.xticks(x, x+1)
plt.yscale('log')
plt.xlabel('r')
plt.ylabel('Absoluter Fehlerwert')
plt.legend()
plt.savefig(f'{base_path}/DEIM_error_split_1.png', bbox_inches='tight', dpi=600)
plt.savefig(f'{base_path}/DEIM_error_split_1.svg', bbox_inches='tight', dpi=600)
plt.show()

In [ ]:
%run -i Functions.py

##### N = 3 Soliton #####
##### Plot Modell- vs Projektionsfehler des POD ROMs #####
r_range = 7
total_error_pod_model = np.zeros(r_range)
total_error_pod_proj = np.zeros(r_range)



##### Räumliche Diskretisierung #####
n = 1000
x_pre = np.linspace(-L2/2,L2/2,n+1)
x = x_pre[:n]

##### Anfangsbedingung (Soliton) #####
N = 3
u0 = N/np.cosh(x)

X_true = Simulate_NLS_FOM(u0, x, t)
X = NLS_POD_Offline_1(u0, x, t)

Psi, Sigma, PhiT = np.linalg.svd(X)

for r in range(1,r_range+1):
    print(f'Fehler wird berechnet für r = {r}')
    Psi_inner = Psi[:,:r]@Psi[:,:r].T
    ops = NLS_POD_Offline_2_3(X, r)
    X_pod = NLS_POD_Online(u0, t, ops)
    total_error_pod_proj[r-1] += np.linalg.norm(X_true[:,tt] - Psi_inner@X_true[:,tt])**2
    total_error_pod_model[r-1] += np.linalg.norm(Psi_inner@X_true[:,tt] - X_pod[:,tt])**2


In [ ]:
##### PLOTTING #####
plt.rcParams.update({'font.size': 12})
plt.figure(figsize=(15,7))
marker = itertools.cycle(('.', '>', 'X', 'D', 'P', 'd', '*'))


x = np.arange(r_range)

plt.plot(x, total_error_pod_model, label = "POD Modelierungsfehler", linestyle='dashed', marker='s')
plt.plot(x, total_error_pod_proj, label = "POD Projektionsfehler", linestyle='dashed', marker='s')

plt.xticks(x, x+1)
plt.yscale('log')
plt.xlabel('r')
plt.ylabel('Absoluter Fehlerwert')
plt.legend()
plt.savefig(f'{base_path}/DEIM_error_split_2.png', bbox_inches='tight', dpi=600)
plt.savefig(f'{base_path}/DEIM_error_split_2.svg', bbox_inches='tight', dpi=600)
plt.show()